In [2]:
from IPython.external import mathjax; mathjax.install_mathjax()

Extracting to C:\Users\Sim\.ipython\nbextensions\mathjax


0

# Asynchronous I/O

여기서는

* Blocking, Non-blocking and asynchronous operations에 대한 설명
* Understanding event loop
* 이것을 지원하는 python 3.4 패키지중의 하나인 asyncio에 대해서 설명 및 예제를 설명합니다.

## Blocking, Non-blocking and asynchronous operations

어떤 함수를 실행시킬때

* 결과를 받을때까지 기다리는게 Blocking
* 준비된 결과까지만을 받는게 Non-blocking, 경험상으로는 일시키고 나는 하던 일 계속하는 것.
* 결과가 준비된 순간에 사용자의 의해서 등록된 처리 함수를 수행하는 것, Asynchronous

## Event Loop

내가 만약 여러 어떤 것들(예를 들어, 여러 파일을 동시에 접근하려고 할때 같은 경우)을 사용하려고 할때, 일반적으로 시스템은 그들을 '실제로' 동시에 접속할 수 없다.

즉, 사용할 준비가 되면, 그때 사용해야 한다. 

이렇게 내가 사용하고자 하는 어떤 Resource가 내가 원하는 준비가 되었다는 정보를 가지고 있고, 사용자는 이 정보를 계속 체크하므로써, 준비된 시점에 사용하는 방식을 'Event Loop'라고 부른다.

Socket Programming에서 서버 구현 방식에서 흔히 쓰이는 패턴이다.

* Echo Server 예제

### Event Loop을 지원하는 API

* select() : 감시할 수 있는 리소스의 개수가 제한되어 있다. O(n)
* poll() : select()함수의 확장판, 리소스의 개수가 꽤 크다. O(n)
* epoll() : poll()함수보다 더 좋다. O(1)으로 찾아서 알려준다. 단 linux에서만 지원한다.
* kqueue() : BSD 또는 Mac OS X의 epoll()함수 버젼이다.

epoll의 설명 : http://www.joinc.co.kr/modules/moniwiki/wiki.php/Site/Network_Programing/AdvancedComm/epoll24

### Event Loop는 꽤 많은데서 사용하고 있다.

* Tornado Web Server 
* Twisted
* asyncio
* Gevent
* Eventlet
* Node.JS

등등

## Asyncio

Python에서 asynchronous programming을 할 수 있도록 지원해주는 패키지이다. 파이썬의 창시자인 귀도 반 로썸과 그 외의 몇명의 참여자로 시작된 패키지이며, Python 3.4부터는 정식 패키지로 지원이 된다.

AsyncIO는 다음과 같은 요소들을 사용하여 프로그래밍 한다.

* Event Loop 
* Coroutine : (제가 생각하기에) 가장 중요한 개념. 정식 문서에서는  "A coroutine is a generator that follows certain conventions." 라고 설명함, 특정 동작이 완료 될때까지 (예를 들어, 디스크에서 지정된 만큼 데이터를 가져올때까지) 기다리고 있다가, 해당 동작이 완료되면, 기다림이 시작된 위치의 다음 라인으로 와서 계속됨 (yield의 동작을 생각해보자!!)
* Future : 이 녀석은 아직 완료되지 않은 처리의 상태를 나타내는 class, 사실 Future는 concurrent 패키지의 Future를 가져와 AsyncIO에 적합하게한 Class이다.
* Tasks : 이것은 coroutine의 스케쥴을 관리하는 클래스. 만일 하나의 coroutine이 waiting상태면, 다른 coroutine이 실행된다.

Reference : https://docs.python.org/3/library/asyncio.html

Reference : https://speakerdeck.com/devunt/python3-dot-4-asyncio

### 백문이 불여일견

In [1]:
import asyncio

@asyncio.coroutine
def sleep_coroutine(f):
    yield from asyncio.sleep(2)
    f.set_result('Done!')
    
future = asyncio.Future()
loop = asyncio.get_event_loop()
loop.run_until_complete(sleep_coroutine(future)) # run event loop

In [3]:
@asyncio.coroutine 
def sleep_coro(name, seconds=1):    
    print("[%s] coroutine will sleep for %d second(s)…" % (name, seconds))    
    yield from asyncio.sleep(seconds)    
    print("[%s] done!" % name) 
    
tasks = [asyncio.Task(sleep_coro('Task-A', 10)),                
         asyncio.Task(sleep_coro('Task-B')),                
         asyncio.Task(sleep_coro('Task-C'))]    
loop.run_until_complete(asyncio.gather(*tasks))

[Task-A] coroutine will sleep for 10 second(s)…
[Task-B] coroutine will sleep for 1 second(s)…
[Task-C] coroutine will sleep for 1 second(s)…
[Task-B] done!
[Task-C] done!
[Task-A] done!


[None, None, None]

In [6]:
import time

@asyncio.coroutine 
def sleep_coro(name, loop, seconds=1):    
    future = loop.run_in_executor(None, time.sleep, seconds)
    print("[%s] coroutine will sleep for %d second(s)…" % (name, seconds))    
    yield from future
    print("[%s] done!" % name) 
    
loop = asyncio.get_event_loop()
tasks = [asyncio.Task(sleep_coro('Task-A', loop, 10)),                
         asyncio.Task(sleep_coro('Task-B', loop)),                
         asyncio.Task(sleep_coro('Task-C', loop))]    
loop.run_until_complete(asyncio.gather(*tasks))
loop.close()

[Task-A] coroutine will sleep for 10 second(s)…
[Task-B] coroutine will sleep for 1 second(s)…
[Task-C] coroutine will sleep for 1 second(s)…
[Task-C] done!
[Task-B] done!
[Task-A] done!
